In [ ]:
from google.colab import driveimport osimport sysfrom pathlib import Path# Montar Google Driveprint("Montando Google Drive...")drive.mount('/content/drive')# Navegar para o projetoPROJECT_ROOT = '/content/drive/MyDrive/planthealth_training'os.chdir(PROJECT_ROOT)sys.path.append(str(PROJECT_ROOT))print(f"✓ Diretório do projeto: {PROJECT_ROOT}")# Criar estruturafolders = ['data', 'data/raw', 'data/processed', 'models', 'models/checkpoints', 'results', 'results/visualizations']for folder in folders:    Path(folder).mkdir(parents=True, exist_ok=True)print("✓ Estrutura criada")# Instalar dependênciasprint("\nInstalando dependências...")!pip install -q tensorflow keras matplotlib seaborn scikit-learn opencv-python-headless pillow pandas numpyprint("\n" + "="*60)print("✅ SETUP COMPLETO!")print("="*60 + "\n")

In [ ]:
import tensorflow as tffrom tensorflow import kerasfrom tensorflow.keras.applications import VGG16from tensorflow.keras.preprocessing.image import ImageDataGeneratorfrom tensorflow.keras.models import Model, Sequentialfrom tensorflow.keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2Dfrom tensorflow.keras.optimizers import Adamfrom tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateauimport matplotlib.pyplot as pltimport seaborn as snsimport numpy as npimport pandas as pdfrom datetime import datetime# Configurar estilosns.set_style("whitegrid")plt.rcParams['figure.figsize'] = (12, 8)# Task 1: Print TensorFlow versionprint("="*60)print("TASK 1: TensorFlow Version")print("="*60)print(f"TensorFlow version: {tf.__version__}")print(f"Keras version: {keras.__version__}")print(f"GPU disponível: {tf.config.list_physical_devices('GPU')}")print("\n")

In [ ]:
print("="*60)print("CONFIGURAÇÕES DE TREINAMENTO OTIMIZADAS")print("="*60)# ⚡ CONFIGURAÇÕES OTIMIZADAS PARA VELOCIDADEIMG_SIZE = (150, 150)  # Reduzido de 224x224 para 150x150 (3x mais rápido!)BATCH_SIZE = 64        # Aumentado de 32 para 64 (2x mais rápido!)EPOCHS = 10            # Para teste inicial (depois pode aumentar para 15)LEARNING_RATE = 0.001NUM_CLASSES = 26       # Seu dataset tem 26 classesprint(f"✓ Tamanho da imagem: {IMG_SIZE}")print(f"✓ Batch size: {BATCH_SIZE}")print(f"✓ Epochs: {EPOCHS}")print(f"✓ Learning rate: {LEARNING_RATE}")print(f"✓ Número de classes: {NUM_CLASSES}")print("\n")

In [ ]:
print("="*60)print("PREPARANDO GERADORES DE DADOS")print("="*60)# Data augmentation SIMPLIFICADO para treino (mais rápido)train_datagen = ImageDataGenerator(    rescale=1./255,    rotation_range=15,      # Reduzido de 30 para 15    width_shift_range=0.1,  # Reduzido de 0.2 para 0.1    height_shift_range=0.1, # Reduzido de 0.2 para 0.1    horizontal_flip=True,    zoom_range=0.1,         # Reduzido de 0.2 para 0.1    validation_split=0.2    # 80% treino, 20% validação)# Apenas rescale para teste (sem augmentation)test_datagen = ImageDataGenerator(    rescale=1./255)# Caminhos dos dadosTRAIN_DIR = '/content/drive/MyDrive/Colab Notebooks/planthealth.ai/data/raw/train'TEST_DIR = '/content/drive/MyDrive/Colab Notebooks/planthealth.ai/data/raw/test'# Criar geradorestrain_generator = train_datagen.flow_from_directory(    TRAIN_DIR,    target_size=IMG_SIZE,    batch_size=BATCH_SIZE,    class_mode='categorical',    subset='training',    shuffle=True)validation_generator = train_datagen.flow_from_directory(    TRAIN_DIR,    target_size=IMG_SIZE,    batch_size=BATCH_SIZE,    class_mode='categorical',    subset='validation',    shuffle=False)# Task 2: Create test generatortest_generator = test_datagen.flow_from_directory(    TEST_DIR,    target_size=IMG_SIZE,    batch_size=BATCH_SIZE,    class_mode='categorical',    shuffle=False)# Task 3: Print length of train generatorprint("\n" + "="*60)print("TASK 3: Informações dos Geradores")print("="*60)print(f"✓ Total de batches no train_generator: {len(train_generator)}")print(f"✓ Total de batches no validation_generator: {len(validation_generator)}")print(f"✓ Total de batches no test_generator: {len(test_generator)}")print(f"✓ Total de imagens de treino: {train_generator.samples}")print(f"✓ Total de imagens de validação: {validation_generator.samples}")print(f"✓ Total de imagens de teste: {test_generator.samples}")print(f"✓ Classes encontradas: {len(train_generator.class_indices)}")print(f"✓ Nomes das classes: {list(train_generator.class_indices.keys())[:5]}...")print("\n")

In [ ]:
print("="*60)print("CONSTRUINDO MODELO COM VGG16 PRÉ-TREINADO")print("="*60)# Carregar VGG16 pré-treinado (sem as camadas fully connected)base_model = VGG16(    weights='imagenet',    include_top=False,    input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))# ⚡ CONGELAR TODAS AS CAMADAS DO VGG16 (MUITO IMPORTANTE!)# Isso torna o treinamento 3-5x mais rápidobase_model.trainable = Falseprint(f"✓ Modelo base VGG16 carregado")print(f"✓ Camadas do VGG16: {len(base_model.layers)}")print(f"✓ Parâmetros treináveis do VGG16: {base_model.trainable}")# Adicionar camadas customizadasx = base_model.outputx = GlobalAveragePooling2D()(x)  # Mais eficiente que Flattenx = Dense(256, activation='relu')(x)x = Dropout(0.5)(x)x = Dense(128, activation='relu')(x)x = Dropout(0.3)(x)predictions = Dense(NUM_CLASSES, activation='softmax')(x)# Criar modelo finalmodel = Model(inputs=base_model.input, outputs=predictions)# Task 4: Print model summaryprint("\n" + "="*60)print("TASK 4: RESUMO DO MODELO")print("="*60)model.summary()print("\n")

In [ ]:
# Task 5: Compile modelprint("="*60)print("TASK 5: COMPILANDO MODELO")print("="*60)model.compile(    optimizer=Adam(learning_rate=LEARNING_RATE),    loss='categorical_crossentropy',    metrics=['accuracy'])print("✓ Modelo compilado com sucesso!")print(f"  - Optimizer: Adam (lr={LEARNING_RATE})")print(f"  - Loss: categorical_crossentropy")print(f"  - Metrics: accuracy")print("\n")

In [ ]:
print("="*60)print("CONFIGURANDO CALLBACKS")print("="*60)# Criar diretório para checkpointscheckpoint_dir = 'models/checkpoints'Path(checkpoint_dir).mkdir(parents=True, exist_ok=True)# Callbackscheckpoint = ModelCheckpoint(    filepath=f'{checkpoint_dir}/best_model.keras',    monitor='val_accuracy',    save_best_only=True,    mode='max',    verbose=1)early_stop = EarlyStopping(    monitor='val_loss',    patience=5,    restore_best_weights=True,    verbose=1)reduce_lr = ReduceLROnPlateau(    monitor='val_loss',    factor=0.5,    patience=3,    min_lr=0.00001,    verbose=1)callbacks = [checkpoint, early_stop, reduce_lr]print("✓ Callbacks configurados:")print("  - ModelCheckpoint: Salva melhor modelo")print("  - EarlyStopping: Para se não melhorar (patience=5)")print("  - ReduceLROnPlateau: Reduz learning rate se estagnado")print("\n")

In [ ]:
print("="*60)print("🚀 INICIANDO TREINAMENTO - FASE 1 (FEATURE EXTRACTION)")print("="*60)print("⚡ Otimizações ativas:")print(f"  - Modelo base CONGELADO (VGG16.trainable = False)")print(f"  - Imagens reduzidas: {IMG_SIZE}")print(f"  - Batch size aumentado: {BATCH_SIZE}")print(f"  - Data augmentation simplificado")print("="*60 + "\n")# Calcular steps por epochSTEPS_PER_EPOCH = len(train_generator)VALIDATION_STEPS = len(validation_generator)# Para teste SUPER RÁPIDO, descomente as linhas abaixo:# STEPS_PER_EPOCH = 100  # Apenas 100 batches por epoch# VALIDATION_STEPS = 50   # Apenas 50 batches de validaçãoprint(f"Steps por epoch: {STEPS_PER_EPOCH}")print(f"Validation steps: {VALIDATION_STEPS}\n")# Treinar!start_time = datetime.now()history = model.fit(    train_generator,    steps_per_epoch=STEPS_PER_EPOCH,    epochs=EPOCHS,    validation_data=validation_generator,    validation_steps=VALIDATION_STEPS,    callbacks=callbacks,    verbose=1)end_time = datetime.now()training_time = (end_time - start_time).total_seconds() / 60print("\n" + "="*60)print("✅ TREINAMENTO CONCLUÍDO!")print("="*60)print(f"⏱️  Tempo total: {training_time:.2f} minutos")print(f"📊 Melhor val_accuracy: {max(history.history['val_accuracy']):.4f}")print(f"📊 Melhor val_loss: {min(history.history['val_loss']):.4f}")print("="*60 + "\n")

In [ ]:
print("="*60)print("SALVANDO MODELO E HISTÓRICO")print("="*60)# Salvar modelo finalmodel.save('models/fruit_classifier_phase1.keras')print("✓ Modelo salvo: models/fruit_classifier_phase1.keras")# Salvar históricoimport jsonwith open('results/history_phase1.json', 'w') as f:    json.dump(history.history, f)print("✓ Histórico salvo: results/history_phase1.json")print("\n")

In [ ]:
# TASK 6: Plotar curvas de accuracyprint("="*60)print("TASK 6: PLOTANDO CURVAS DE ACCURACY")print("="*60)plt.figure(figsize=(12, 5))# Accuracyplt.subplot(1, 2, 1)plt.plot(history.history['accuracy'], label='Train Accuracy', marker='o')plt.plot(history.history['val_accuracy'], label='Validation Accuracy', marker='s')plt.title('Model Accuracy - Phase 1 (Feature Extraction)', fontsize=14, fontweight='bold')plt.xlabel('Epoch')plt.ylabel('Accuracy')plt.legend(loc='lower right')plt.grid(True, alpha=0.3)# Lossplt.subplot(1, 2, 2)plt.plot(history.history['loss'], label='Train Loss', marker='o')plt.plot(history.history['val_loss'], label='Validation Loss', marker='s')plt.title('Model Loss - Phase 1 (Feature Extraction)', fontsize=14, fontweight='bold')plt.xlabel('Epoch')plt.ylabel('Loss')plt.legend(loc='upper right')plt.grid(True, alpha=0.3)plt.tight_layout()plt.savefig('results/visualizations/phase1_training_curves.png', dpi=300, bbox_inches='tight')plt.show()print("✓ Gráficos salvos: results/visualizations/phase1_training_curves.png\n")

In [ ]:
# TASK 7: Plotar curvas de loss detalhadasprint("="*60)print("TASK 7: PLOTANDO CURVAS DE LOSS DETALHADAS")print("="*60)plt.figure(figsize=(10, 6))plt.plot(history.history['loss'], label='Training Loss', linewidth=2, marker='o', markersize=4)plt.plot(history.history['val_loss'], label='Validation Loss', linewidth=2, marker='s', markersize=4)plt.title('Training and Validation Loss - Phase 1', fontsize=16, fontweight='bold')plt.xlabel('Epoch', fontsize=12)plt.ylabel('Loss', fontsize=12)plt.legend(loc='upper right', fontsize=11)plt.grid(True, alpha=0.3)plt.tight_layout()plt.savefig('results/visualizations/phase1_loss_curves.png', dpi=300, bbox_inches='tight')plt.show()print("✓ Gráfico de loss salvo: results/visualizations/phase1_loss_curves.png\n")

In [ ]:
print("="*60)print("AVALIANDO MODELO NO CONJUNTO DE TESTE")print("="*60)test_loss, test_accuracy = model.evaluate(test_generator, verbose=1)print("\n" + "="*60)print("📊 RESULTADOS NO TESTE:")print("="*60)print(f"Test Loss: {test_loss:.4f}")print(f"Test Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")print("="*60 + "\n")

In [ ]:
print("="*60)print("🚀 FASE 2: FINE-TUNING (Descongelando camadas finais)")print("="*60)# Descongelar as últimas 4 camadas do VGG16 para fine-tuningbase_model.trainable = Truefine_tune_at = len(base_model.layers) - 4# Congelar todas as camadas exceto as últimas 4for layer in base_model.layers[:fine_tune_at]:    layer.trainable = Falseprint(f"✓ Total de camadas no VGG16: {len(base_model.layers)}")print(f"✓ Camadas congeladas: {fine_tune_at}")print(f"✓ Camadas descongeladas para fine-tuning: {len(base_model.layers) - fine_tune_at}")# Recompilar com learning rate MENOR para fine-tuningmodel.compile(    optimizer=Adam(learning_rate=LEARNING_RATE/10),  # 10x menor!    loss='categorical_crossentropy',    metrics=['accuracy'])print(f"✓ Modelo recompilado com lr={LEARNING_RATE/10} (10x menor)")print("\n")# Treinar mais alguns epochsprint("Iniciando fine-tuning...\n")history_fine = model.fit(    train_generator,    steps_per_epoch=STEPS_PER_EPOCH,    epochs=5,  # Menos epochs para fine-tuning    validation_data=validation_generator,    validation_steps=VALIDATION_STEPS,    callbacks=callbacks,    verbose=1)print("\n" + "="*60)print("✅ FINE-TUNING CONCLUÍDO!")print("="*60 + "\n")# Salvar modelo fine-tunedmodel.save('models/fruit_classifier_phase2_finetuned.keras')print("✓ Modelo fine-tuned salvo: models/fruit_classifier_phase2_finetuned.keras\n")

In [ ]:
# TASK 8: Plotar curvas do modelo fine-tunedprint("="*60)print("TASK 8: PLOTANDO CURVAS DO MODELO FINE-TUNED")print("="*60)# Combinar históricostotal_accuracy = history.history['accuracy'] + history_fine.history['accuracy']total_val_accuracy = history.history['val_accuracy'] + history_fine.history['val_accuracy']total_loss = history.history['loss'] + history_fine.history['loss']total_val_loss = history.history['val_loss'] + history_fine.history['val_loss']plt.figure(figsize=(14, 6))# Accuracyplt.subplot(1, 2, 1)plt.plot(total_accuracy, label='Train Accuracy', linewidth=2)plt.plot(total_val_accuracy, label='Validation Accuracy', linewidth=2)plt.axvline(x=len(history.history['accuracy'])-1, color='red', linestyle='--', label='Fine-tuning start')plt.title('Complete Training: Accuracy', fontsize=14, fontweight='bold')plt.xlabel('Epoch')plt.ylabel('Accuracy')plt.legend()plt.grid(True, alpha=0.3)# Lossplt.subplot(1, 2, 2)plt.plot(total_loss, label='Train Loss', linewidth=2)plt.plot(total_val_loss, label='Validation Loss', linewidth=2)plt.axvline(x=len(history.history['loss'])-1, color='red', linestyle='--', label='Fine-tuning start')plt.title('Complete Training: Loss', fontsize=14, fontweight='bold')plt.xlabel('Epoch')plt.ylabel('Loss')plt.legend()plt.grid(True, alpha=0.3)plt.tight_layout()plt.savefig('results/visualizations/complete_training_curves.png', dpi=300, bbox_inches='tight')plt.show()print("✓ Gráficos salvos: results/visualizations/complete_training_curves.png\n")

In [ ]:
print("="*60)print("AVALIAÇÃO FINAL DO MODELO FINE-TUNED")print("="*60)test_loss_final, test_accuracy_final = model.evaluate(test_generator, verbose=1)print("\n" + "="*60)print("📊 RESULTADOS FINAIS NO TESTE:")print("="*60)print(f"Test Loss: {test_loss_final:.4f}")print(f"Test Accuracy: {test_accuracy_final:.4f} ({test_accuracy_final*100:.2f}%)")print("\n📈 Melhoria após fine-tuning:")print(f"  Accuracy: {test_accuracy:.4f} → {test_accuracy_final:.4f} (+{(test_accuracy_final-test_accuracy)*100:.2f}%)")print("="*60 + "\n")

In [ ]:
# TASK 9: Extrair features e visualizarprint("="*60)print("TASK 9: EXTRAINDO FEATURES COM MODELO BASE")print("="*60)# Criar modelo para extração de features (somente VGG16)feature_extractor = Model(inputs=base_model.input, outputs=base_model.output)# Pegar uma imagem de testetest_images, test_labels = next(test_generator)sample_image = test_images[0:1]  # Primeira imagemsample_label = np.argmax(test_labels[0])# Extrair featuresfeatures = feature_extractor.predict(sample_image)print(f"✓ Shape das features extraídas: {features.shape}")print(f"✓ Classe real da imagem: {sample_label}")# Visualizarfig, axes = plt.subplots(1, 2, figsize=(12, 5))# Imagem originalaxes[0].imshow(sample_image[0])axes[0].set_title(f'Test Image (Class: {sample_label})', fontsize=12, fontweight='bold')axes[0].axis('off')# Feature map (primeira camada de features)feature_map = features[0, :, :, 0]  # Primeiro canalim = axes[1].imshow(feature_map, cmap='viridis')axes[1].set_title('Extracted Features (VGG16)', fontsize=12, fontweight='bold')axes[1].axis('off')plt.colorbar(im, ax=axes[1])plt.tight_layout()plt.savefig('results/visualizations/task9_feature_extraction.png', dpi=300, bbox_inches='tight')plt.show()print("✓ Visualização salva: results/visualizations/task9_feature_extraction.png\n")

In [ ]:
# TASK 10: Predição com modelo fine-tunedprint("="*60)print("TASK 10: PREDIÇÃO COM MODELO FINE-TUNED")print("="*60)# Fazer prediçãoprediction = model.predict(sample_image)predicted_class = np.argmax(prediction[0])confidence = prediction[0][predicted_class] * 100# Obter nomes das classesclass_names = list(train_generator.class_indices.keys())print(f"✓ Classe real: {class_names[sample_label]}")print(f"✓ Classe predita: {class_names[predicted_class]}")print(f"✓ Confiança: {confidence:.2f}%")# Visualizar prediçãofig, axes = plt.subplots(1, 2, figsize=(14, 5))# Imagemaxes[0].imshow(sample_image[0])axes[0].set_title(f'Test Image\nTrue: {class_names[sample_label]}',                   fontsize=12, fontweight='bold')axes[0].axis('off')# Top 5 prediçõestop5_idx = np.argsort(prediction[0])[-5:][::-1]top5_probs = prediction[0][top5_idx] * 100top5_classes = [class_names[i] for i in top5_idx]axes[1].barh(range(5), top5_probs, color='steelblue')axes[1].set_yticks(range(5))axes[1].set_yticklabels(top5_classes)axes[1].set_xlabel('Confidence (%)', fontsize=11)axes[1].set_title('Top 5 Predictions (Fine-tuned Model)', fontsize=12, fontweight='bold')axes[1].grid(axis='x', alpha=0.3)# Destacar predição corretaif predicted_class == sample_label:    axes[1].get_children()[0].set_color('green')plt.tight_layout()plt.savefig('results/visualizations/task10_finetuned_prediction.png', dpi=300, bbox_inches='tight')plt.show()print("✓ Visualização salva: results/visualizations/task10_finetuned_prediction.png\n")

In [ ]:
print("\n" + "="*80)print("🎉 PROJETO CONCLUÍDO COM SUCESSO!")print("="*80)print("\n📊 RESUMO DOS RESULTADOS:")print("-" * 80)print(f"Fase 1 (Feature Extraction):")print(f"  - Test Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")print(f"  - Test Loss: {test_loss:.4f}")print(f"\nFase 2 (Fine-tuning):")print(f"  - Test Accuracy: {test_accuracy_final:.4f} ({test_accuracy_final*100:.2f}%)")print(f"  - Test Loss: {test_loss_final:.4f}")print(f"  - Melhoria: +{(test_accuracy_final-test_accuracy)*100:.2f}%")print("\n✅ TODAS AS TASKS COMPLETADAS:")print("-" * 80)print("  ✓ Task 1: TensorFlow version impresso")print("  ✓ Task 2: Test generator criado")print("  ✓ Task 3: Length do train generator impresso")print("  ✓ Task 4: Model summary impresso")print("  ✓ Task 5: Modelo compilado")print("  ✓ Task 6: Curvas de accuracy plotadas (Fase 1)")print("  ✓ Task 7: Curvas de loss plotadas (Fase 1)")print("  ✓ Task 8: Curvas do modelo fine-tuned plotadas")print("  ✓ Task 9: Features extraídas e visualizadas")print("  ✓ Task 10: Predição com modelo fine-tuned")print("\n⚡ OTIMIZAÇÕES APLICADAS:")print("-" * 80)print("  ✓ Imagens reduzidas: 224x224 → 150x150 (3x mais rápido)")print("  ✓ Batch size aumentado: 32 → 64 (2x mais rápido)")print("  ✓ Modelo base congelado na Fase 1 (5x mais rápido)")print("  ✓ Data augmentation simplificado")print("  ✓ Callbacks para early stopping e learning rate adaptativo")print("\n" + "="*80)print("🚀 Treinamento otimizado e funcional!")print("="*80 + "\n")